In [1]:
import sys
sys.path.insert(0, "/Users/larryshamalama/Documents/GitHub/pymc3")

import numpy as np
import pymc3 as pm
import arviz as az

from aesara import tensor as at
import aesara

You are running the v4 development version of PyMC3 which currently still lacks key features. You probably want to use the stable v3 instead which you can either install via conda or find on the v3 GitHub branch: https://github.com/pymc-devs/pymc3/tree/v3


In [2]:
X = np.array([-1])
K = 20

def stick_breaking(betas):
    portion_remaining = at.concatenate([[1], (1 - betas[:-1])])
    
    return at.mul(betas, at.cumprod(portion_remaining))

### Testing GP submodule

Sampling from a GP prior

In [3]:
with pm.Model() as model:
    gp = pm.gp.Latent()
    gp.prior("gp-prior", X)
    trace = pm.sample(1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [gp-prior_rotated_]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 18 seconds.


### Trying to sample from a DP Prior

In [4]:
with pm.Model() as model:
    alpha = pm.Gamma("alpha", 1, 1)
    betas = pm.Beta("betas", 1, alpha, shape=K)
    w = pm.Deterministic("w", stick_breaking(betas))
    
    base_dist = pm.Normal("G0", mu=0, sigma=1, shape=K)
    
    dp = pm.Deterministic("dp", at.sum(at.mul(w, at.lt(X, base_dist)), axis=0))
    
    trace = pm.sample(
        draws=1000,
        chains=1,
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, betas, G0]


/Users/larryshamalama/Documents/GitHub/pymc3/pymc3/step_methods/hmc/integration.py:115: RuntimeWarning: invalid value encountered in double_scalars
  energy = kinetic - logp
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 12 seconds.
There were 30 divergences after tuning. Increase `target_accept` or reparameterize.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [5]:
with pm.Model() as model:
    base_dist = pm.Normal("G0", 0, 1, shape=(K,))
    dp = pm.dp.DirichletProcess(
        name="dp",
        alpha=1,
        base_dist=base_dist,
        K=K,
    )
    dp.prior("prior", X)
    
    trace = pm.sample(
        draws=1000,
        chains=1,
    )

AttributeError: module 'pymc3' has no attribute 'dp'